In [54]:
# TODO : support multiple hypotheses

## imports

In [55]:
import os
import numpy as np
from datetime import datetime
from tqdm import tqdm
from copy import deepcopy
from utils import multi_exp_data_to_fig

## constants

In [56]:
# hypothesis = 'pesto_protein_scores_ablation'
# hypothesis = 'optimizer_batchsize_lr_gridsearch'
# hypothesis = 'pts_without_mlp_b'
# hypothesis = 'pts_deep_encoder_mlps'
# hypothesis = 'pts_shallow_encoder_mlps'
# hypothesis = 'pts_shallow_encoder_mlps_curated_dataset'
# hypothesis = 'patch_attention'
# hypothesis = 'patch_attention_transition'
hypothesis = 'patch_attention_masking'
evaluation_type = 'validation'  # can be train / validation / test

## fetch data

In [57]:
experiments = os.listdir(f'../configurations/data/{hypothesis}')
experiments = [e.split('.json')[0] for e in experiments]

In [58]:
# Iterate over all experiments, per experiment - sort runs by time
# Going from most recent to oldest, take the first run of the expeirment that finished running (has inference results over all 5 folds)

run_paths = []
for experiment in experiments:
    try:
        experiment_runs = os.listdir(f'../results/patch_to_score/hypotheses/{hypothesis}/{experiment}')
    except Exception as e:
        print(e)
        continue
    try:
        experiment_runs = sorted(experiment_runs, key=lambda x: datetime.strptime(x.split('_')[0], '%Y-%m-%d'), reverse=True)
    except Exception as e:
        print(e)
    experiment_valid_run_found = False
    for updated_experiment_run in experiment_runs:
        run_path = f'{hypothesis}/{experiment}/{updated_experiment_run}'
        if 'fold_4' in os.listdir(f'../results/patch_to_score/hypotheses/{run_path}'):
            run_paths.append(run_path)
            experiment_valid_run_found = True
            break
    if not experiment_valid_run_found:
        print(f'experiment: {experiment} didnt finish, skipping')
len(run_paths)

12

In [59]:
base_paths = [f'../results/patch_to_score/hypotheses/{path}' for path in run_paths]

## parse data

In [60]:
multi_exp_data = dict()
for base_path in tqdm(base_paths):
    exp_data = []
    exp_name = base_path.split('/')[-2]
    for fold in range(5):
        try:
            fold_labels = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'labels.npy'))
            fold_predictions = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'predictions.npy'))[:, 0]
            exp_data.append(np.stack([fold_labels, fold_predictions]))
        except Exception as e:
            pass
    multi_exp_data[exp_name] = exp_data

100%|██████████| 12/12 [00:00<00:00, 212.78it/s]


## filter

In [61]:
# def name_modification(name: str) -> str:
#     res = name.split('_')
#     res = f'{res[1]}_{res[2]}_{res[3]}_{res[4]}_{res[6]}_{res[7]}_{res[8]}_{res[9]}_{res[11]}_{res[13]}_{res[14].split("-")[0]}_{res[17]}_{res[19]}_{res[20].split("-")[0]}'
#     return res

# def name_modification(name: str) -> str:
#     res = name.replace('hidden_sizes_mlp_', '')
#     res = res.replace('hidden_sizes_mlp_', '')
#     return res

def name_modification(name: str) -> str:
    return name

In [62]:
filtered_multi_exp_data = deepcopy(multi_exp_data)
filtered_multi_exp_data = {name_modification(exp_name): exp_data for exp_name, exp_data in filtered_multi_exp_data.items()}
# filtered_multi_exp_data = {exp_name: exp_data for exp_name, exp_data in filtered_multi_exp_data.items() if ('sgd' not in exp_name and '0.0001' not in exp_name)}

## plot

In [63]:
fig, exp_to_auc = multi_exp_data_to_fig(filtered_multi_exp_data, 
                            show_std_in_legend=False, log_scale=True, resolution=1_000)
# fig.show()

computing rocs


100%|██████████| 12/12 [00:00<00:00, 35.74it/s]

adding random roc
fig update


In [64]:
sorted_experiments = sorted(exp_to_auc.items(), key=lambda x: x[1], reverse=True)
sorted_experiments

[('dim_256-heads_1-dropout_25', 0.9158376350672048),
 ('dim_256-heads_1-dropout_10', 0.9154346101403353),
 ('dim_512-heads_1-dropout_25', 0.913853181223066),
 ('dim_512-heads_1-dropout_10', 0.9127558450363932),
 ('dim_512-heads_8-dropout_25', 0.9116894609597971),
 ('dim_512-heads_8-dropout_10', 0.9113189280886939),
 ('dim_256-heads_8-dropout_25', 0.910315542718814),
 ('dim_256-heads_8-dropout_10', 0.9088508311321956),
 ('dim_512-heads_4-dropout_10', 0.9015375004326739),
 ('dim_256-heads_4-dropout_25', 0.9004469322369058),
 ('dim_256-heads_4-dropout_10', 0.8999823907855123),
 ('dim_512-heads_4-dropout_25', 0.8976436062884853)]